# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [22]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [23]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [24]:
class Flatten(nn.Module):
    def forward(self,x):
        return x.view(x.shape[0],-1)

class Reshape(nn.Module):
    def forward(self,x):
        return x.view(-1,1,28,28)

LEARNINGRATE=0.9

def init_weights(layer):
    if type(layer)==nn.Linear or type(layer)==nn.Conv2d:
        torch.nn.init.xavier_uniform_(layer.weight)

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            Reshape(),
            nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1,padding=2),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            Flatten(),
            nn.Linear(in_features=16*5*5,out_features=120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
            )
        self.net.apply(init_weights)

    def forward(self,x):
        x=self.net(x)
        return x
    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=LEARNINGRATE,momentum=0.1)

Next, we can start to train and evaluate!

In [25]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss=torch.tensor([0.0],dtype=torch.float32)
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        model.net.train()
        
        optimizer.zero_grad()
        pred=model.forward(images)
        loss=criterion(pred,labels)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            train_loss+=loss.float()


    # evaluate
    # TODO:calculate the accuracy using training and testing dataset
    train_num=0
    train_accuracy=torch.tensor([0.0],dtype=torch.float32)
    for images,labels in train_loader:
        with torch.no_grad():
            labels=labels.long()
            train_pred=model.forward(images)
            train_accuracy+=(torch.sum(torch.argmax(train_pred,dim=1)==labels)).float()     
            train_num+=train_pred.shape[0]

    test_num=0
    test_accuracy=torch.tensor([0.0],dtype=torch.float32)
    for images, labels in test_loader:  
        model.net.eval()
        with torch.no_grad():
            labels=labels.long()
            test_pred=model.forward(images)
            test_accuracy+=(torch.sum(torch.argmax(test_pred,dim=1)==labels)).float()     
            test_num+=test_pred.shape[0]
    print('Epoch %d, Loss: %0.4f, Training accuracy: %0.2f%%, Testing accuracy: %0.2f%%' % (epoch+1,train_loss/train_num,train_accuracy*100/train_num,test_accuracy*100/test_num))
    time.sleep(1)

100%|██████████| 468/468 [00:53<00:00,  8.80it/s]
Epoch 1, Loss: 0.0088, Training accuracy: 94.44%, Testing accuracy: 94.41%
100%|██████████| 468/468 [00:52<00:00,  8.90it/s]
Epoch 2, Loss: 0.0010, Training accuracy: 96.95%, Testing accuracy: 96.88%
100%|██████████| 468/468 [00:53<00:00,  8.78it/s]
Epoch 3, Loss: 0.0006, Training accuracy: 97.53%, Testing accuracy: 97.35%
100%|██████████| 468/468 [00:53<00:00,  8.78it/s]
Epoch 4, Loss: 0.0005, Training accuracy: 98.51%, Testing accuracy: 98.29%
100%|██████████| 468/468 [00:53<00:00,  8.83it/s]
Epoch 5, Loss: 0.0004, Training accuracy: 98.67%, Testing accuracy: 98.49%
100%|██████████| 468/468 [00:57<00:00,  8.20it/s]
Epoch 6, Loss: 0.0003, Training accuracy: 99.05%, Testing accuracy: 98.57%
100%|██████████| 468/468 [00:54<00:00,  8.58it/s]
Epoch 7, Loss: 0.0003, Training accuracy: 99.05%, Testing accuracy: 98.62%
100%|██████████| 468/468 [00:56<00:00,  8.28it/s]
Epoch 8, Loss: 0.0003, Training accuracy: 99.20%, Testing accuracy: 98.87%


#### Q5:
Please print the training and testing accuracy.